In [1]:
import os
import io
import cv2
import copy
import math
import wandb
import random
import numpy as np
import pickle as pkl
from collections import deque
from tqdm import tqdm, trange
from typing import Deque, Dict, List, Tuple
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader, random_split



## dataset

In [12]:

name_corridor = ['0717_1411', 
                '0720_1129',
                '0722_1428',
                '0722_1437',
                '0722_1448',
                '0722_1500',
                '0722_1506',
                '0722_1510',
                '0722_1514',
                '0724_1101',
                '0724_1132',
                '0724_1204',
                '0727_1005',
                '0727_1027',
                '0727_1102',
                '0727_1435',
                '0727_1452',
                '0727_1510',
                '0727_1520',
                '0805_1108',
                '0805_1127',
                '0805_1147']

name_parking = ['0717_1504',
                '0720_1105',
                '0805_1349',
                '0805_1425']

paths_corridor, paths_parking = [], []

main_path = '/media/ray/intelSSD/mmdemo_train'
dirs = os.listdir(main_path)
dirs.sort()
for d in dirs:
    dirs1 = os.listdir(main_path+'/'+d)
    dirs1.sort()
    dirs2 = os.listdir(main_path+'/'+d+'/'+dirs1[1])
    dirs2.sort()
    for d2 in dirs2:
        if any(s in d2 for s in name_corridor):
            paths_corridor.append(main_path+'/'+d+'/'+dirs1[1]+'/'+d2)
#             print(paths_corridor[-1])
        elif any(s in d2 for s in name_parking):
            paths_parking.append(main_path+'/'+d+'/'+dirs1[1]+'/'+d2)
#             print(paths_parking[-1])
            
print('%d episodes'%len(paths_corridor))
print('%d episodes'%len(paths_parking))


153 episodes
31 episodes


In [13]:
class MMDataset(Dataset):
    def __init__(self, paths):
        self.transitions = []

        for p in tqdm(paths):
            with open(p, "rb") as f:
                demo = pkl.load(f, encoding="bytes")
                self.transitions.extend(demo)
        
    def __getitem__(self,index):
        mm_scan = self.transitions[index][b'mm_scan']
        laser_scan = self.transitions[index][b'laser_scan']
        mm_scan = torch.Tensor(mm_scan).reshape(1,-1)
        laser_scan = torch.Tensor(laser_scan).reshape(1,-1)
        
        return mm_scan, laser_scan
    
    def __len__(self):
        return len(self.transitions)

    
batch_size = 128

corridor_dataset = MMDataset(paths_corridor)
loader_corridor = DataLoader(dataset=corridor_dataset,
                    batch_size=batch_size,
                    shuffle=True,
                    num_workers=4)

parking_dataset = MMDataset(paths_parking)
loader_parking = DataLoader(dataset=parking_dataset,
                    batch_size=batch_size,
                    shuffle=True,
                    num_workers=4)


100%|██████████| 31/31 [00:00<00:00, 95.11it/s]


## hyper parameters

In [16]:
hyper_parameter = dict(
    kernel=3,
    stride=2,
    padding=2,
    latent=128,
    deconv_dim=32,
    deconv_channel=128,
    adjust_linear=235,
    epoch=100,
    learning_rate=0.001,
)
class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)
config = Struct(**hyper_parameter)

## model

In [17]:
class MMvae(nn.Module):
    def __init__(self):
        super(MMvae, self).__init__()
        kernel = 3
        stride = 2
        self.conv = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=kernel, stride=stride),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=kernel, stride=stride),
            nn.ReLU()
        )
        
        dim = 64*59
        self.linear1=nn.Sequential(
            nn.Linear(dim,512),
            nn.ReLU()
        )
        self.en_fc1=nn.Linear(512,config.latent)
        self.en_fc2=nn.Linear(512,config.latent)
        
        self.de_fc1=nn.Sequential(
            nn.Linear(config.latent,config.deconv_channel*config.deconv_dim),
            nn.ReLU()
        )
        
        self.de_conv =nn.Sequential(
            nn.ConvTranspose1d(config.deconv_channel, config.deconv_channel//2, kernel, stride=stride, padding=config.padding),
#             nn.ReLU(),
            nn.ConvTranspose1d(config.deconv_channel//2, config.deconv_channel//4, kernel, stride=stride, padding=config.padding),
#             nn.ReLU(),
            nn.ConvTranspose1d(config.deconv_channel//4, 1, kernel, stride=stride, padding=config.padding),
#             nn.ReLU(),
        )
        self.adjust_linear=nn.Sequential(
            nn.Linear(config.adjust_linear,241),
            nn.ReLU()
        )

        
    def encoder(self,x):
        x = self.conv(x)
        x = x.view(x.size(0),-1)
        x = self.linear1(x)
        mean = self.en_fc1(x)
        logvar = self.en_fc2(x)
        return mean, logvar

    def reparameter(self, mean, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mean + eps*std

    def decoder(self,x):
        x = self.de_fc1(x)
        x = x.view(-1, config.deconv_channel, config.deconv_dim)
        x = self.de_conv(x)
        x = self.adjust_linear(x)
        return x

    def forward(self,x):
        mean, logvar = self.encoder(x)
        x = self.reparameter(mean, logvar)
        x = self.decoder(x)
        return x ,mean ,logvar

## test

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device, ',device)
model = MMvae()
model.to(device)
model_path = "/home/ray/subt-analyze/07-mmwave-vae/generate_lidar/wandb/run-20200726_155759-3235857x/model.pth"
model.load_state_dict(torch.load(model_path))

device,  cuda:0


<All keys matched successfully>

## L1

In [22]:
L1 = []
t = tqdm(loader_corridor)
for mm_scan, laser_scan in t:
    mm_scan = mm_scan.to(device)
    
    x_hat ,mean ,logvar = model(mm_scan)
    x_hat = x_hat.detach().cpu().numpy().reshape(mm_scan.size(0),-1)
    laser_scan = laser_scan.numpy().reshape(mm_scan.size(0),-1)
    mm_scan = mm_scan.detach().cpu().numpy().reshape(mm_scan.size(0),-1)
    
    l1 = np.mean(np.abs(laser_scan-x_hat))
    L1.append(l1)
    
print("\n corridor L1: %.4f"%np.mean(L1))

100%|██████████| 582/582 [00:01<00:00, 475.46it/s]


 corridor L1: 0.2315


In [23]:
L1 = []
t = tqdm(loader_parking)
for mm_scan, laser_scan in t:
    mm_scan = mm_scan.to(device)
    
    x_hat ,mean ,logvar = model(mm_scan)
    x_hat = x_hat.detach().cpu().numpy().reshape(mm_scan.size(0),-1)
    laser_scan = laser_scan.numpy().reshape(mm_scan.size(0),-1)
    mm_scan = mm_scan.detach().cpu().numpy().reshape(mm_scan.size(0),-1)
    
    l1 = np.mean(np.abs(laser_scan-x_hat))
    L1.append(l1)
    
print("\n parking L1: %.4f"%np.mean(L1))

100%|██████████| 120/120 [00:00<00:00, 374.38it/s]


 parking L1: 0.2722
